In [ ]:
# Subset mutations for Sellers lab
# only include CCLE2 lines

import pandas as pd
import dalmatian as dm

In [ ]:
ccle2_ws = "fccredits-silver-tan-7621/CCLE_v2"

In [ ]:
ccle2wm = dm.WorkspaceManager(ccle2_ws).disable_hound()

In [ ]:
ccle2samples = ccle2wm.get_samples()

In [ ]:
ccle2samples.index

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
merged_somatic_mutations = tc.get(name='mutations-latest-ed72', version=24, file='merged_somatic_mutations')

In [ ]:
merged_somatic_mutations

In [ ]:
subset_mut = merged_somatic_mutations[merged_somatic_mutations.DepMap_ID.isin(ccle2samples.index)]
subset_mut

In [ ]:
subset_mut.to_csv("ccle2_mutations.csv", index=None)

In [ ]:
merged_segments_all = tc.get(name='cn-latest-d8d4', version=26, file='merged_segments_all')


In [ ]:
merged_segments_all

In [ ]:
GBM_names = ["42MGBA", "8MGBA", "A172", "AM38", "BECKER", "CAS1", "CCFSTTG1", "DBTRG05MG", "DKMG", "GAMG", "GB1", "GI1", "GMS10", "GOS3", "HS683", "KALS1", "KG1C", "KNS42", "KNS60", "KNS81", "LN18", "LN229", "M059K", "NMCG1", "SF126", "SF268", "SF295", "SF539", "SNB75", "SNU1105", "SNU201", "SNU466", "SNU489", "SNU626", "SNU738", "SW1088", "SW1783", "T98G", "TM31", "U118MG", "U251MG", "U87MG", "YH13", "YKG1"]

In [ ]:
from depmapomics import tracker as track
trackerobj = track.initTracker()
t = trackerobj.read_tracker()
t

In [ ]:
GBM_rows = t[t.stripped_cell_line_name.isin(GBM_names)]
GBM_rows

In [ ]:
len(GBM_rows.arxspan_id.unique())

In [ ]:
GBM_seg = merged_segments_all[merged_segments_all.DepMap_ID.isin(GBM_rows.arxspan_id.unique())]
GBM_seg

In [ ]:
GBM_seg.to_csv("ccle_seg.csv", index=None)

In [ ]:
# Check cclf twist snv output
twistwm = dm.WorkspaceManager("nci-mimoun-bi-org/PANCAN_TWIST copy").disable_hound()

In [ ]:
twist_pairsets = twistwm.get_pair_sets()
twist_pairsets

In [ ]:
twist41snv = pd.read_csv(twist_pairsets.loc["CCLF_TWIST41_pairs", "aggregate_snvs"], sep="\t")
twist41snv

In [ ]:
twist41subset = twist41snv[twist41snv.external_id == "ACH-003025_BOM_P7_3D_1"]
twist41subset[twist41subset.Chromosome == "16"]['Hugo_Symbol']

In [ ]:
twist41subset[twist41subset.Chromosome == "16"][['Start_position', 'End_position']]

In [ ]:
twist41snv[twist41snv.external_id == "ACH-003025_BOM_P7_3D_1"].to_csv("ACH003025.csv", index=None)

In [ ]:
maf = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/1cdc902e-234f-450c-9303-406fb34ff6ad/WGS_pipeline/b6bc83be-af09-4ac2-8c91-57bf070b8870/call-filterMaf/CDS-0JJKl3_outMAFfn.txt", sep="\t")
maf


In [ ]:
maf.dbNSFP_Polyphen2_HVAR_pred.unique()

In [ ]:
proteincoding_genes_tpm_logp1 = tc.get(name='expression-d035', version=21, file='proteincoding_genes_tpm_logp1')
proteincoding_genes_tpm_logp1

In [ ]:
proteincoding_genes_tpm_logp1[]

In [ ]:
filter_col = [col for col in proteincoding_genes_tpm_logp1 if col.startswith(('ERBB2 (', 'ESR1 (', 'PGR ('))]
filter_col

In [ ]:
from scipy.stats import zscore
zscores = proteincoding_genes_tpm_logp1[filter_col].apply(zscore)
zscores.loc[['ACH-002949', 'ACH-002950', 'ACH-002951', 'ACH-002883', 'ACH-002885']]